# ¡Escala tus *pipelines* de procesado de datos mixtos sin salir de scikit-learn!

## Pedro Morales, Data Scientist @ Ebury
<part.morales@gmail.com>

![ebury](img/ebury_logo.jpg)
![pycones](img/pycones.png)

### PyConES 2018 @ Málaga, 2018-10-06

# ¿Por qué scikit-learn?

![sklearn](img/sklearn_logo.png)

* Extensa colección de algoritmos y herramientas auxiliares para transformar datos y construir modelos
* API muy expresiva y notación altamente estandarizada
* Permite fácil integración de funcionalidades personalizadas
* Proyecto muy activo en constante mejora

* *[Inserte su motivo]*
* **Python**!


# `Pipeline` - ¿Para qué?

* Antes de entrar al estimador o modelo final, los datos pueden requerir alguna(s) de las siguientes operaciones:
  - Limpieza
  - Reducción de dimensiones
  - Generación de nuevas variables o *features* 
  
* `sklearn.pipeline.Pipeline` nos ayuda a encapsular cadenas de transformaciones **secuenciales** sobre nuestros datos


In [1]:
# Ejemplo: Titanic dataset

import pandas as pd

data = pd.read_csv((
    'https://raw.githubusercontent.com/amueller/'
    'scipy-2017-sklearn/091d371/notebooks/datasets/titanic3.csv')
)

data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [2]:
columns = ['fare', 'age']
target = 'survived'

X = data[columns].values
y = data[target].values

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline(steps=[('imputer', SimpleImputer()),
                      ('scaler', StandardScaler()),
                      ('classifier', LogisticRegression(solver='lbfgs'))])

In [4]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(clf, X, y, cv=5)
print('CV accuracy: %0.3f' % cv_scores.mean())
print('"0" class prior: %0.3f' % (1 - y.mean()))

CV accuracy: 0.633
"0" class prior: 0.618


# ¿Y si el procesado no es *secuencial*?

* En el caso de querer incorporar **diferentes tipos de datos** al modelo, el procesado consiste en varias cadenas de operaciones **paralelas**, no en una única rama secuencial
* Incluso si las variables son del mismo tipo, no implica que deban seguir la misma cadena de transformaciones